HSE, Applied Time Series Forecasitng , Winter 2023-24

<font color="green"> Lesson #13: Bonus Topics </font>

<span style="color:black; font-size: 12pt"></span>

Alexey Romanenko,
<font color="blue">alexromsput@gmail.com</font>

**Key words:**
 demand restoration, value from forecasting

**Your feedback:**  please provide you feedback  <a href="https://forms.gle/bEV5iwmTqdM9rUoy8"> here </a>

In [3]:
import numpy as np
import pandas as pd
from copy import deepcopy

# don't forget to upload utils file to the colab session
import warnings, pylab

warnings.filterwarnings('ignore')

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

# Using TS forecasting packages. Online Demo

## Online Demo based on HW3

In [8]:
# загружаем данные
ts_hw3 = pd.read_csv('train.csv', sep=',')
ts_hw3['period_start_dt'] = pd.to_datetime(ts_hw3['period_start_dt'], format= "%Y-%m-%d") #dayfirst=True)
ts_hw3.set_index(['period_start_dt'], inplace=True)
ts_hw3 = ts_hw3.sort_index() # must have
ts_hw3.head()

,Unnamed: 0,product_rk,store_location_rk,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
period_start_dt,,,,,,,,,,
2016-12-19,0,40369,309,29.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21361,40369,1034,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21362,40370,1034,31.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21363,40372,1034,11.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21364,40373,1034,14.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# transform data to original format
df_actual = ts_hw3.reset_index().rename(columns = {'demand':'metricValue', 'period_start_dt':'date'})
df_actual['metric']= 'demand'
df_actual['status']= 'actual'

# df_actual[['date', 'metric', 'metricValue', 'product_rk', 'store_location_rk', 'status']]

# конвертим дату в datetime и сортируем данные по дате
df_actual['date'] = pd.to_datetime(df_actual['date'])
df_actual = df_actual.sort_values('date')

# определяем поля, в которых зашит id одного ts
groupby_columns = ['product_rk', 'store_location_rk']
# в этом датасете product_id - константа для всех ts

In [18]:
# find largest ts (in terms of sum of target variable)
_list_ts = df_actual.groupby(groupby_columns)['metricValue'].mean().sort_values(ascending = False)
_list_ts[:5]

,,metricValue
product_rk,store_location_rk,
40370,309,105.000000
40369,309,68.666667
40370,1281,64.000604
40372,309,50.000000
40370,425,42.622383


In [20]:
# extract 5
_list_ts[:5].reset_index()[groupby_columns]

,product_rk,store_location_rk
0,40370,309
1,40369,309
2,40370,1281
3,40372,309
4,40370,425


In [ ]:
# рисуем графиики каждого из ts, чтобы увидеть скрытые компоненты ts (тренд и сезонность)
# можно делать stl - декомпозицию, но как правило все можно увидеть на простом графике

# analysis of top-5 ts
df_actual = _list_ts[:5]


def plot_by_ts(df, groupby_columns, target_col = 'metricValue', date_col_name = 'date', granularity='D', aggregation_method = 'sum'):
    # from plotly.subplots import make_subplots
    pd.options.plotting.backend = "plotly"

    plot_df = df.set_index(date_col_name).groupby(groupby_columns).resample(granularity).agg({target_col:aggregation_method}).reset_index().set_index([date_col_name] + groupby_columns)[target_col]
    # plot_df = df.set_index([date_col_name] + groupby_columns)[[target_col]]

    # транспонируем все group_by колонки:
    for i in range(len(groupby_columns)):
        plot_df = plot_df.unstack()
    # print(plot_df.head())

    # делаем название колонок "плоскими"
    plot_df.columns = [' '.join(col).strip() for col in plot_df.columns.values]

    # рисуем графики
    plot_df.plot().show()
    return

plot_by_ts(df_actual['product_rk' = 40370, 'store_location_rk'==309], groupby_columns, granularity = 'W')

### Вывод:
### Сезонность наблюдается: скорее во всех ts
### Тренд наблюдается Черновики заявки - Казань, Черновики заявки - Уфа

### предлается брать additivную компонету никаких ярких трендов не видно,  (пердлагается взять простое ES или для учета плохо заметно недельной несезонности сезонную модель ES)

## Using of python packages for time series forecasting

https://github.com/aromanenko/ATSF/wiki/List-of-packages-for-Time-Series-Forecasting-in-Python

# Business Value from Forecasting
 Business case: your company launched new Demand Forecasting System. What couldn't be as a business-value due to this implementation?
  
  a) free money due less overforecasting or less underforecasting
  
  b) free resources of analytics (less cost of support resources)
  
  c) increase transparancy and consistency in decision making processes (new money will come)
  
  d) speed up decision making process (more money due to fast reaction, more frequent decision making process)